In [1]:
## https://stackoverflow.com/questions/15285691/googlemaps-api-address-to-coordinates-latitude-longitude
import urllib
import urllib.request
import simplejson
import re

googleGeocodeUrl = 'http://maps.googleapis.com/maps/api/geocode/json?'

def get_coordinates(query, from_sensor=False):
    query = query.encode('utf-8')
    params = {
        'address': query,
        'sensor': "true" if from_sensor else "false"
    }
    url = googleGeocodeUrl + urllib.parse.urlencode(params)
    json_response = urllib.request.urlopen(url)
    response = simplejson.loads(json_response.read())
    if response['results']:
        location = response['results'][0]['geometry']['location']
        latitude, longitude = location['lat'], location['lng']
        print(query.decode('utf8'), latitude, longitude)
    else:
        latitude, longitude = None, None
        print(query.decode('utf8'), "<no results>")
    return latitude, longitude

In [2]:
get_coordinates("市政府")

市政府 25.0375417 121.5644327


(25.0375417, 121.5644327)

In [3]:
get_coordinates("桃園")

桃園 24.9936281 121.3009798


(24.9936281, 121.3009798)

In [4]:
lon1, lat1 = get_coordinates("桃園")
lon2, lat2 = get_coordinates("市政府")

桃園 24.9936281 121.3009798
市政府 25.0375417 121.5644327


In [5]:
#https://developers.google.com/maps/documentation/distance-matrix/intro?hl=zh-tw#RequestParameters
#https://stackoverflow.com/questions/17267807/python-google-maps-driving-time
def drivetime(lon1, lat1, lon2, lat2):
    orig_coord = lon1, lat1
    dest_coord = lon2, lat2
    url = "http://maps.googleapis.com/maps/api/distancematrix/json?origins={0}&destinations={1}&mode=driving&language=en-EN&sensor=false".format(re.sub("[() ]", "", str(orig_coord)),re.sub("[() ]", "", str(dest_coord)))
    print(url)
    result= simplejson.load(urllib.request.urlopen(url))
    driving_time = result['rows'][0]['elements'][0]['duration']['value']
    return(driving_time)

In [6]:
drivetime(lon1, lat1, lon2, lat2) #returns secs

http://maps.googleapis.com/maps/api/distancematrix/json?origins=24.9936281,121.3009798&destinations=25.0375417,121.5644327&mode=driving&language=en-EN&sensor=false


2352

In [7]:
from urllib.request import quote
def drivetime_q(from_q, dest_q):
    url = "http://maps.googleapis.com/maps/api/distancematrix/json?origins={0}&destinations={1}&mode=driving&language=en-EN&sensor=false".format(quote(from_q), quote(dest_q))
    #print(url)
    try:
        result= simplejson.load(urllib.request.urlopen(url))
        driving_time = result['rows'][0]['elements'][0]['duration']['value']
        return(driving_time)
    except KeyError:
        return(999999999)

In [8]:
drivetime_q("北投區", "松山區")

1685

In [9]:
drivetime_q("金門", "松山區")

999999999

In [ ]:
import os
import pandas as pd
os.chdir("..")
os.getcwd()

In [ ]:
#ttl_places.csv
data=pd.read_clipboard(header=None).values

In [ ]:
data[1:]

In [ ]:
import itertools
def expandgrid(*itrs):
   product = list(itertools.product(*itrs))
   return {'Var{}'.format(i+1):[x[i] for x in product] for i in range(len(itrs))}


In [ ]:
places = expandgrid(data[1:], data[1:])

In [ ]:
places = pd.DataFrame(places)

In [ ]:
places = places.assign(cardrive_sec=999999999)
places = places[places['Var1']!=places['Var2']]
places = places.reset_index(drop=True)

In [ ]:
from random import randint
import time

for i in range(0, places.shape[0]+1):
    places['cardrive_sec'][i] = drivetime(places['Var1'].str[0][i], places['Var2'].str[0][i])
    print (round(i/places.shape[0]*100,3), "%", end="\r")
    #time.sleep(randint(1, 4))